In [1]:
import polars as pl 
import sys
from pathlib import Path
import importlib
import matplotlib.pyplot as plt

# Set the font family to Arial
# https://g.co/gemini/share/3898a74b2d77
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.sans-serif'] = ['Arial'] # Add Arial to the sans-serif list

plt.rcParams.update({
    'font.size': 20,
})

REFERENCE_GENOME = "hg38"
TILE_SIZE = 1000
METH_READ_PHASED_DIR = Path('/scratch/ucgd/lustre-labs/quinlan/data-shared/dna-methylation/CEPH1463.GRCh38.hifi.founder-phased.all-cpgs')

REPO_DIR = Path("/scratch/ucgd/lustre-labs/quinlan/u6018199/tapestry")
sys.path.append(str(REPO_DIR / "src" / "util"))
sys.path.append(f"{REPO_DIR}/src") 

pl.Config.set_tbl_rows(10) 

polars.config.Config

# Compute difference of haplotype-specific methylations for all samples 

In [2]:
# [TESTING] 2 mins to run for 2 samples
cmd = f"python analysis/imprinting.py --reference_genome {REFERENCE_GENOME} --tile_size {TILE_SIZE} --meth_read_phased_dir {METH_READ_PHASED_DIR} --output_bed analysis/delta_meth_all_samples.bed"
testing_cmd = f"PYTHONPATH=src/util {cmd} --testing"
testing_cmd

'PYTHONPATH=src/util python analysis/imprinting.py --reference_genome hg38 --tile_size 1000 --meth_read_phased_dir /scratch/ucgd/lustre-labs/quinlan/data-shared/dna-methylation/CEPH1463.GRCh38.hifi.founder-phased.all-cpgs --output_bed analysis/delta_meth_all_samples.bed --testing'

In [3]:
# [Production] 25 mins to run for all samples
production_cmd = f"PYTHONPATH=src/util nohup {cmd} > analysis/delta_meth_all_samples.log 2>&1 &" 
production_cmd

'PYTHONPATH=src/util nohup python analysis/imprinting.py --reference_genome hg38 --tile_size 1000 --meth_read_phased_dir /scratch/ucgd/lustre-labs/quinlan/data-shared/dna-methylation/CEPH1463.GRCh38.hifi.founder-phased.all-cpgs --output_bed analysis/delta_meth_all_samples.bed > analysis/delta_meth_all_samples.log 2>&1 &'

In [4]:
from read_data import read_dataframe_from_bed

def read_all_samples(bed): 
    df = read_dataframe_from_bed(bed)
    return df

DF_DELTA_METH_ALL_SAMPLES = read_all_samples('delta_meth_all_samples.bed')
print(len(DF_DELTA_METH_ALL_SAMPLES))
DF_DELTA_METH_ALL_SAMPLES.sample(10)

2903221


chrom,start,end,NA12877_num_cpgs,NA12877_num_valid_cpgs,NA12877_num_valid_cpgs_pat,NA12877_num_valid_cpgs_mat,NA12877_count,NA12877_model,NA12878_num_cpgs,NA12878_num_valid_cpgs,NA12878_num_valid_cpgs_pat,NA12878_num_valid_cpgs_mat,NA12878_count,NA12878_model,NA12887_num_cpgs,NA12887_num_valid_cpgs,NA12887_num_valid_cpgs_pat,NA12887_num_valid_cpgs_mat,NA12887_count,NA12887_model,NA12885_num_cpgs,NA12885_num_valid_cpgs,NA12885_num_valid_cpgs_pat,NA12885_num_valid_cpgs_mat,NA12885_count,NA12885_model,NA12883_num_cpgs,NA12883_num_valid_cpgs,NA12883_num_valid_cpgs_pat,NA12883_num_valid_cpgs_mat,NA12883_count,NA12883_model,NA12881_num_cpgs,NA12881_num_valid_cpgs,NA12881_num_valid_cpgs_pat,NA12881_num_valid_cpgs_mat,…,200087_model,200101_num_cpgs,200101_num_valid_cpgs,200101_num_valid_cpgs_pat,200101_num_valid_cpgs_mat,200101_count,200101_model,200102_num_cpgs,200102_num_valid_cpgs,200102_num_valid_cpgs_pat,200102_num_valid_cpgs_mat,200102_count,200102_model,200103_num_cpgs,200103_num_valid_cpgs,200103_num_valid_cpgs_pat,200103_num_valid_cpgs_mat,200103_count,200103_model,200104_num_cpgs,200104_num_valid_cpgs,200104_num_valid_cpgs_pat,200104_num_valid_cpgs_mat,200104_count,200104_model,200105_num_cpgs,200105_num_valid_cpgs,200105_num_valid_cpgs_pat,200105_num_valid_cpgs_mat,200105_count,200105_model,200106_num_cpgs,200106_num_valid_cpgs,200106_num_valid_cpgs_pat,200106_num_valid_cpgs_mat,200106_count,200106_model
str,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,…,f64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,f64,f64
"""chr2""",146759000,146760000,1,1,1,1,-0.02,0.021,1,1,0,0,null,null,1,1,0,0,null,null,1,1,1,1,-0.132,-0.153,1,1,1,1,0.05,0.035,1,1,0,0,…,null,1,1,0,0,null,null,1,1,0,0,null,null,1,1,1,1,-0.12,-0.146,1,1,1,1,-0.205,-0.031,1,1,1,1,0.089,0.005,1,1,0,0,null,null
"""chr15""",59640000,59641000,7,7,7,7,0.078143,0.021,6,6,6,6,-0.007667,-0.010833,6,6,6,6,-0.026667,0.027333,6,6,6,6,0.027167,0.071833,6,6,6,6,0.023333,0.0275,6,6,6,6,…,null,7,7,6,7,-0.02,0.035857,6,6,6,6,-0.030667,0.033167,7,7,0,0,null,null,6,6,6,6,0.015167,0.002833,6,6,6,6,0.02,0.074333,7,7,0,0,null,null
"""chr5""",138334000,138335000,13,13,13,13,-0.040308,-0.021308,13,13,13,13,-0.001077,0.010923,13,13,13,13,0.038385,0.022769,13,13,13,13,0.040308,0.018077,13,13,13,13,0.088077,0.010538,13,13,13,1,…,null,13,13,13,13,0.044154,0.024769,13,13,13,13,0.016462,0.010923,13,13,13,13,0.001077,0.005846,13,13,13,13,0.006923,0.035692,13,13,13,13,-0.066462,-0.029231,13,13,13,13,0.095308,0.048385
"""chr14""",70985000,70986000,12,12,12,12,0.041667,0.037917,12,12,12,12,-0.007833,0.006917,12,12,12,12,-0.022333,0.009833,13,13,13,13,-0.009154,0.000769,13,13,13,13,0.037769,0.019615,12,12,12,12,…,null,12,12,12,12,-0.0045,0.005583,12,12,12,12,0.02825,0.021583,12,12,12,12,-0.053667,0.013417,12,12,12,12,-0.049,-0.046917,12,12,12,12,0.040083,0.050667,12,12,12,12,0.020667,-0.002833
"""chr19""",54194000,54195000,13,13,13,13,-0.008231,0.004385,13,13,13,13,-0.080769,-0.061385,13,13,13,13,0.016385,-0.014692,13,13,13,13,-0.011,0.007538,13,13,13,13,-0.023923,0.024769,13,0,0,0,…,null,13,13,13,13,0.041308,0.067154,13,13,13,13,-0.01,-0.034615,13,13,0,13,null,null,13,13,0,1,null,null,13,13,13,13,-0.026,-0.026846,13,13,0,0,null,null
"""chr14""",44217000,44218000,11,11,11,11,0.013727,0.009091,11,11,11,11,0.021273,0.019909,11,11,11,11,0.036636,0.040545,11,11,11,11,0.037091,0.047818,11,11,11,11,0.050727,-0.017182,11,11,11,11,…,0.028,11,11,11,11,-0.020364,-0.017364,11,11,11,11,-0.024455,0.028091,11,11,11,11,-0.059,0.013273,11,11,11,11,-0.006909,0.017909,11,11,11,11,0.014727,-0.017091,11,11,11,11,0.073091,0.073182
"""chrX""",77745000,77746000,6,6,0,0,null,null,6,6,6,6,-0.005667,0.056833,6,6,6,6,-0.264,-0.225167,6,6,0,0,null,null,6,6,0,0,null,null,6,6,0,0,…,null,6,6,0,0,null,null,6,6,0,0,null,null,6,6,0,0,null,null,6,6,

# Developing an imprinted-locus caller by inspecting candidates in IGV and bed files for various threshold values

IGV snapshots for the examples below can be found at https://docs.google.com/presentation/d/1A95YqjHMGJnEQdnCw4BQknhhEqAl9EUROupWtPXfZRw/edit?usp=sharing 

### Thresholding difference in methylation between haplotypes averaged over each tile

In [74]:
import imprinting
importlib.reload(imprinting)
from imprinting import call_imprinted_loci

call_imprinted_loci(
    DF_DELTA_METH_ALL_SAMPLES, 
    meth_mode='model',
    delta_meth_threshold=0.9, 
    num_valid_cpgs_per_hap_threshold=0,
    valid_cpg_ratio_threshold=0.0
).sample(10, seed=42) 

Number of candidate imprinted loci: 82


chrom,start,end,imprinted_samples,num_imprinted_samples,locus
str,i64,i64,list[str],u32,str
"""chr13""",98602000,98603000,"[""200105""]",1,"""chr13:98602000-98603000"""
"""chr6""",32454000,32455000,"[""NA12881"", ""NA12882"", … ""200105""]",4,"""chr6:32454000-32455000"""
"""chr19""",56839000,56840000,"[""NA12877"", ""200081""]",2,"""chr19:56839000-56840000"""
"""chr12""",65142000,65143000,"[""NA12883"", ""NA12879"", … ""200105""]",4,"""chr12:65142000-65143000"""
"""chr14""",39697000,39698000,"[""200082""]",1,"""chr14:39697000-39698000"""
"""chr10""",93686000,93687000,"[""200082"", ""200085"", … ""200106""]",5,"""chr10:93686000-93687000"""
"""chr3""",111525000,111526000,"[""200105""]",1,"""chr3:111525000-111526000"""
"""chr11""",2700000,2701000,"[""NA12877"", ""NA12878"", ""200105""]",3,"""chr11:2700000-2701000"""
"""chr5""",69949000,69950000,"[""200085"", ""200086""]",2,"""chr5:69949000-69950000"""


In [ ]:
# this could be filtered out by setting num_valid_cpgs_per_hap_threshold = 1 AND valid_cpg_ratio_threshold = 0.9 (say)
! bash inspect_cpgs.sh 200105 chr13:98602000-98603000 

#chrom  start_cpg  end_cpg   total_read_count  methylation_level_count  total_read_count_pat  total_read_count_mat  methylation_level_pat_count  methylation_level_mat_count  cpg_overlaps_at_least_one_snv  snv_genotypes  cpg_is_allele_specific
chr13   98602000   98602002  43                0.465                    22                    21                    0.909                        0.0                          true                           hom            false
chr13   98602006   98602008  43                0.465                    22                    21                    0.909                        0.0                          false                          .              false
chr13   98602012   98602014  22                1.0                      22                                          1.0                                                       false                          .              false
chr13   98602044   98602046  22                0.818                    22     

In [ ]:
# this could be filtered out by setting num_valid_cpgs_per_hap_threshold = 1 (say)
! bash inspect_cpgs.sh 200105 chr6:32454000-32455000

#chrom  start_cpg  end_cpg   total_read_count  methylation_level_count  total_read_count_pat  total_read_count_mat  methylation_level_pat_count  methylation_level_mat_count  cpg_overlaps_at_least_one_snv  snv_genotypes  cpg_is_allele_specific
chr6    32454283   32454285  42                0.429                    18                    24                    1.0                          0.0                          true                           het            true
chr6    32454346   32454348                                                                                                                                                   true                           hom            false
chr6    32454357   32454359  41                0.39                     18                    23                    0.0                          0.696                        true                           het            true
chr6    32454412   32454414                                                      

In [ ]:
# this is a real imprinted locus
! bash inspect_cpgs.sh NA12877 chr19:56839000-56840000 

#chrom  start_cpg  end_cpg   total_read_count  methylation_level_count  total_read_count_pat  total_read_count_mat  methylation_level_pat_count  methylation_level_mat_count  cpg_overlaps_at_least_one_snv  snv_genotypes  cpg_is_allele_specific
chr19   56839003   56839005  89                0.506                    49                    40                    0.14300000000000002          0.9500000000000001           false                          .              false
chr19   56839010   56839012  89                0.528                    49                    40                    0.204                        0.925                        false                          .              false
chr19   56839016   56839018  89                0.551                    49                    40                    0.204                        0.975                        false                          .              false
chr19   56839018   56839020  89                0.562                    49     

In [ ]:
# this could be filtered out by setting num_valid_cpgs_per_hap_threshold = 1 (say)
! bash inspect_cpgs.sh 200082 chr14:39697000-39698000

#chrom  start_cpg  end_cpg   total_read_count  methylation_level_count  total_read_count_pat  total_read_count_mat  methylation_level_pat_count  methylation_level_mat_count  cpg_overlaps_at_least_one_snv  snv_genotypes  cpg_is_allele_specific
chr14   39697210   39697212  28                0.536                    10                    18                    0.0                          0.833                        true                           het            true
chr14   39697695   39697697  28                0.5710000000000001       11                    17                    0.364                        0.706                        false                          .              false
chr14   39697748   39697750  27                0.48100000000000004      11                    16                    0.0                          0.813                        true                           het            true


In [27]:
# It doesn't matter that these exists no CpG at which both meth levels are non-null, 
# as we actually average meth levels on each haplotype BEFORE computing the difference of the averages.
# This could be filtered out by setting valid_cpg_ratio_threshold = 0.9 (say)
! bash inspect_cpgs.sh 200105 chr3:111525000-111526000

#chrom  start_cpg  end_cpg    total_read_count  methylation_level_count  total_read_count_pat  total_read_count_mat  methylation_level_pat_count  methylation_level_mat_count  cpg_overlaps_at_least_one_snv  snv_genotypes  cpg_is_allele_specific
chr3    111524998  111525000  20                0.2                      12                                          0.0                                                       true                           het            true
chr3    111525020  111525022  20                0.45                     12                                          0.75                                                      true                           het            true
chr3    111525039  111525041  20                0.25                     12                                          0.0                                                       true                           het            true
chr3    111525098  111525100  20                0.25                     12   

In [ ]:
# this is real 
! bash inspect_cpgs.sh 200105 chr11:2700000-2701000

#chrom  start_cpg  end_cpg  total_read_count  methylation_level_count  total_read_count_pat  total_read_count_mat  methylation_level_pat_count  methylation_level_mat_count  cpg_overlaps_at_least_one_snv  snv_genotypes  cpg_is_allele_specific
chr11   2700006    2700008  34                0.618                    17                    17                    0.294                        0.941                        false                          .              false
chr11   2700012    2700014  34                0.618                    17                    17                    0.23500000000000001          1.0                          false                          .              false
chr11   2700017    2700019  34                0.618                    17                    17                    0.294                        0.941                        false                          .              false
chr11   2700027    2700029  34                0.588                    17          

### Additionally thresholding number of CpGs in each tile

In [75]:
import imprinting
importlib.reload(imprinting)
from imprinting import call_imprinted_loci

call_imprinted_loci(
    DF_DELTA_METH_ALL_SAMPLES, 
    meth_mode='model',
    delta_meth_threshold=0.86, 
    num_valid_cpgs_per_hap_threshold=1,
    valid_cpg_ratio_threshold=0.5
).sample(10, seed=42) 

Number of candidate imprinted loci: 212


chrom,start,end,imprinted_samples,num_imprinted_samples,locus
str,i64,i64,list[str],u32,str
"""chr18""",80148000,80149000,"[""NA12877"", ""NA12881"", … ""200106""]",4,"""chr18:80148000-80149000"""
"""chr5""",95213000,95214000,"[""200101""]",1,"""chr5:95213000-95214000"""
"""chrX""",85700000,85701000,"[""NA12887"", ""NA12885"", ""NA12881""]",3,"""chrX:85700000-85701000"""
"""chr14""",45950000,45951000,"[""NA12881"", ""200081"", ""200106""]",3,"""chr14:45950000-45951000"""
"""chr18""",29097000,29098000,"[""200105""]",1,"""chr18:29097000-29098000"""
"""chr11""",2700000,2701000,"[""NA12877"", ""NA12878"", … ""200106""]",17,"""chr11:2700000-2701000"""
"""chr2""",212373000,212374000,"[""NA12877""]",1,"""chr2:212373000-212374000"""
"""chr12""",11083000,11084000,"[""NA12877"", ""NA12878"", … ""200087""]",8,"""chr12:11083000-11084000"""
"""chr3""",192602000,192603000,"[""NA12883""]",1,"""chr3:192602000-192603000"""


In [ ]:
# this could be filtered with num_valid_cpgs_per_hap_threshold=2
! bash inspect_cpgs.sh 200101 chr5:95213000-95214000

#chrom  start_cpg  end_cpg   total_read_count  methylation_level_count  total_read_count_pat  total_read_count_mat  methylation_level_pat_count  methylation_level_mat_count  cpg_overlaps_at_least_one_snv  snv_genotypes  cpg_is_allele_specific
chr5    95213118   95213120  77                0.26                     31                    46                    0.645                        0.0                          true                           het            true
chr5    95213309   95213311  78                0.359                    32                    46                    0.875                        0.0                          false                          .              false
chr5    95213417   95213419  77                0.312                    31                    46                    0.774                        0.0                          false                          .              false
chr5    95213544   95213546  78                0.551                    32      

In [ ]:
# this could be filtered with num_valid_cpgs_per_hap_threshold=2
! bash inspect_cpgs.sh NA12885 chrX:85700000-85701000

#chrom  start_cpg  end_cpg   total_read_count  methylation_level_count  total_read_count_pat  total_read_count_mat  methylation_level_pat_count  methylation_level_mat_count  cpg_overlaps_at_least_one_snv  snv_genotypes  cpg_is_allele_specific
chrX    85700339   85700341  82                0.36600000000000005      39                    43                    0.0                          0.698                        false                          .              false


In [ ]:
# this could be filtered with num_valid_cpgs_per_hap_threshold=2
! bash inspect_cpgs.sh 200106 chr14:45950000-45951000

#chrom  start_cpg  end_cpg   total_read_count  methylation_level_count  total_read_count_pat  total_read_count_mat  methylation_level_pat_count  methylation_level_mat_count  cpg_overlaps_at_least_one_snv  snv_genotypes  cpg_is_allele_specific
chr14   45950772   45950774  36                0.36100000000000004      18                    18                    0.0                          0.7220000000000001           false                          .              false


In [ ]:
# this could be filtered with num_valid_cpgs_per_hap_threshold=2
! bash inspect_cpgs.sh 200105 chr18:29097000-29098000

#chrom  start_cpg  end_cpg   total_read_count  methylation_level_count  total_read_count_pat  total_read_count_mat  methylation_level_pat_count  methylation_level_mat_count  cpg_overlaps_at_least_one_snv  snv_genotypes  cpg_is_allele_specific
chr18   29097071   29097073  27                0.33299999999999996      14                    13                    0.0                          0.6920000000000001           true                           het            true
chr18   29097082   29097084  27                0.519                    14                    13                    0.28600000000000003          0.7690000000000001           false                          .              false
chr18   29097635   29097637  25                0.4                      13                    12                    0.0                          0.833                        true                           het            true


### More aggressively thresholding number of CpGs in each tile

In [76]:
import imprinting
importlib.reload(imprinting)
from imprinting import call_imprinted_loci

call_imprinted_loci(
    DF_DELTA_METH_ALL_SAMPLES, 
    meth_mode='model',
    delta_meth_threshold=0.76, 
    num_valid_cpgs_per_hap_threshold=5,
    valid_cpg_ratio_threshold=0.5
).sample(10, seed=42) 

Number of candidate imprinted loci: 207


chrom,start,end,imprinted_samples,num_imprinted_samples,locus
str,i64,i64,list[str],u32,str
"""chrX""",49235000,49236000,"[""200081""]",1,"""chrX:49235000-49236000"""
"""chr9""",42296000,42297000,"[""NA12884""]",1,"""chr9:42296000-42297000"""
"""chrX""",131822000,131823000,"[""200081""]",1,"""chrX:131822000-131823000"""
"""chrX""",30713000,30714000,"[""200081""]",1,"""chrX:30713000-30714000"""
"""chrX""",49191000,49192000,"[""200081""]",1,"""chrX:49191000-49192000"""
"""chr20""",30899000,30900000,"[""NA12877""]",1,"""chr20:30899000-30900000"""
"""chr4""",19520000,19521000,"[""NA12887"", ""NA12885"", … ""200106""]",11,"""chr4:19520000-19521000"""
"""chr20""",58840000,58841000,"[""NA12877"", ""NA12878"", … ""200106""]",18,"""chr20:58840000-58841000"""
"""chr7""",23490000,23491000,"[""NA12877"", ""NA12882"", … ""200105""]",10,"""chr7:23490000-23491000"""


In [ ]:
# this could be filtered by computing mean and std of unphased methylation in tile, and requiring that mean is within std of 0.5 
! bash inspect_cpgs.sh NA12884 chr9:42296000-42297000

#chrom  start_cpg  end_cpg   total_read_count  methylation_level_count  total_read_count_pat  total_read_count_mat  methylation_level_pat_count  methylation_level_mat_count  cpg_overlaps_at_least_one_snv  snv_genotypes  cpg_is_allele_specific
chr9    42296144   42296146  77                0.16899999999999998      16                    58                    0.688                        0.0                          true                           het            true
chr9    42296348   42296350  75                0.227                    14                    58                    1.0                          0.0                          true                           hom            false
chr9    42296410   42296412  75                0.2                      14                    58                    0.929                        0.0                          false                          .              false
chr9    42296728   42296730  75                0.147                    13      

In [ ]:
# this could be filtered by removing tiles containing "dirty" read alignments, as the SNV genotypes are homozygous where the alignments suggest heteroyzgous 
! bash inspect_cpgs.sh NA12885 chr4:19520000-19521000

#chrom  start_cpg  end_cpg   total_read_count  methylation_level_count  total_read_count_pat  total_read_count_mat  methylation_level_pat_count  methylation_level_mat_count  cpg_overlaps_at_least_one_snv  snv_genotypes  cpg_is_allele_specific
chr4    19520005   19520007  57                0.316                    33                    24                    0.0                          0.75                         true                           hom            false
chr4    19520021   19520023  55                0.273                    32                    23                    0.0                          0.652                        true                           hom            false
chr4    19520033   19520035  38                0.105                    32                                          0.0                                                       false                          .              false
chr4    19520045   19520047  36                0.055999999999999994     31     